In [2]:
# Project beginning

import torch
import torchvision.models as models
resnet34 = models.resnet34(weights= "IMAGENET1K_V1")

In [3]:
dummy_input = torch.randn(1, 3, 600, 224)
output = resnet34(dummy_input)
print(output.shape)

torch.Size([1, 1000])


In [4]:
# Print the full model architecture
# print(resnet34)

# # List all named children (main blocks)
# for name, module in resnet34.named_children():
#     print(f"{name}: {module}")

In [5]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PetDataset(Dataset):
    def __init__(self, data_dir, split_file, classes, train=True):
        """
        data_dir/
          images/*.jpg
          annotations/trainval.txt
          annotations/test.txt
        split_file: path to trainval.txt or test.txt
        classes: list of breed names in order (1–37)
        """
        self.img_dir = os.path.join(data_dir, "images")
        self.samples = []  # list of (img_path, label)
        for line in open(split_file):
            name, classID, species, _ = line.split()
            label = int(classID) - 1         # breed label 0–36
            self.samples.append((f"{name}.jpg", label))
        # simple 224×224 resize + norm
        self.tfms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485,0.456,0.406], 
                std=[0.229,0.224,0.225]
            )
        ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fname, label = self.samples[idx]
        img = Image.open(os.path.join(self.img_dir, fname)).convert("RGB")
        return self.tfms(img), label

# Usage:
data_dir = "data"
breed_names = sorted({ line.split()[0].rsplit("_",1)[0]
                      for line in open(os.path.join(data_dir,"annotations","trainval.txt")) })
train_ds = PetDataset(data_dir, 
                      split_file=os.path.join(data_dir,"annotations","trainval.txt"),
                      classes=breed_names)
test_ds  = PetDataset(data_dir, 
                      split_file=os.path.join(data_dir,"annotations","test.txt"),
                      classes=breed_names)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
test_dl  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=4)

FileNotFoundError: [Errno 2] No such file or directory: 'data/annotations/trainval.txt'

In [ ]:
train_dl